In [1]:
import numpy as np
import os
import imageio
from multiprocessing import Pool
import cv2
import datetime

from matplotlib.pyplot import imshow

from PIL import Image

from tqdm.notebook import trange, tqdm
#fDataset = 'f00'
fDataset = 'f01'
outPath = "/home/ubuntuos/WeatherUNet/WeatherTiles/scripts/output/cropped_hrrr/"+str(fDataset)+"/"
path = "/home/ubuntuos/WeatherUNet/WeatherTiles/scripts/output/hrrr/"+str(fDataset)+"/"

In [2]:
def getUncroppedImages(X, Y):
    files= os.listdir(path)
    images = []
    for file in files:
        if file.endswith(".png") and not os.path.isfile(os.path.join(outPath, "(" + str(X) + ", " + str(Y) + ")/", file[:-4]+"_(" + str(X) + ", " + str(Y) + ").png")) :
            images.append(file)
    return(images)

def cropImages(image_path):
    input_path = os.path.join(path, image_path)
    try:
        image = cv2.imread(input_path, cv2.IMREAD_UNCHANGED)
        for i in range(4):
            for j in range(4):
                cropped = image[0+256*i:256+256*i, 0+256*j:256+256*j]
                fullpath = os.path.join(outPath, "("+str(j)+", " + str(i) +")/", image_path[:-4]+"_("+str(j)+", " + str(i) +").png")            
                imageio.imwrite(fullpath, cropped)
    except:
        date = datetime.datetime.strptime(image_path, 'CONUS_%Y%m%d-%H0000.png')
        name="hrrr/"+str(fDataset)+date.strftime("/%Y/%m/%d/")+"hrrr.t"+date.strftime("%H")+"z.wrfsfc"+str(fDataset)+".grib2"

        with open('hrrr_processed_images.txt') as f:
            lines = np.array(f.read().splitlines())
            index = np.argwhere(lines == str(name))
            lines = np.delete(lines, index)
            np.savetxt("hrrr_processed_images.txt", lines, fmt='%s')
        os.remove(input_path)
        
if __name__ == '__main__':
    images = getUncroppedImages(3, 3)
    with Pool() as p:
        list(tqdm(p.imap(cropImages, images), desc='Images', total=len(images)))

In [3]:
def getCroppedImages(X=None, Y=None):
    files= []
    images = []
    if not X and not Y:
        for i in range(9):
            for j in range(9):
                files.extend(os.listdir(os.path.join(outPath, "(" + str(i) + ", " + str(j) + ")/")))
    else:
        files.extend(os.listdir(os.path.join(outPath, "(" + str(X) + ", " + str(Y) + ")/")))

    for file in files:
        if file.endswith(".png"):
            images.append(file)
    return(images)

def removeIncompleteImages(image_path):
    input_path = os.path.join(outPath + "(" + image_path[-6] + ", " + image_path[-9] + ")/", image_path)

    image = np.array(cv2.imread(input_path, cv2.IMREAD_GRAYSCALE))
    if (image.shape != (256, 256)):
        os.remove(input_path)
        print(input_path)

if __name__ == '__main__':
    images = getCroppedImages(3,3)
    with Pool() as p:
        list(tqdm(p.imap(removeIncompleteImages, images), desc='Images', total=len(images)))